In [53]:
import pandas as pd
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import platform

# 1. 파일 설정
IMAGE_FILE = 'images.png'
CSV_FILE = 'cancer.csv'

# 데이터 로드
df = pd.read_csv(CSV_FILE)
df_filtered = df[(df['성별'] == '남녀전체') & (df['암종'] == '모든암') & (df['연령군'] == '연령전체')]
df_filtered = df_filtered[df_filtered['발생연도'].astype(str).str.len() == 4].sort_values('발생연도')

years_data = df_filtered['발생연도'].astype(int).tolist()
rates_data = df_filtered['조발생률'].tolist()

# 기준점 설정
min_rate_baseline = rates_data[0]
max_rate_peak = max(rates_data)
if max_rate_peak == min_rate_baseline: max_rate_peak += 1 

# 2. 이미지 처리
img_cv = cv2.imread(IMAGE_FILE)
img_h, img_w = img_cv.shape[:2]
img_gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

# 원본 선 따내기 & 얇게 보정
_, original_line_mask = cv2.threshold(img_gray, 240, 255, cv2.THRESH_BINARY_INV)
kernel = np.ones((3, 3), np.uint8)
thinner_line_mask = cv2.erode(original_line_mask, kernel, iterations=1)

# 실루엣 좌표 구하기
contours, _ = cv2.findContours(original_line_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
main_contour = max(contours, key=cv2.contourArea)
x, y, w, h_sil = cv2.boundingRect(main_contour)

# 3. 캔버스 설정
canvas_w = int(img_w * 1.5)
canvas_h = int(img_h * 1.5) 
offset_x = (canvas_w - img_w) // 2
offset_y = int(img_h * 0.25)

canv_y_top = y + offset_y
canv_y_bottom = y + h_sil + offset_y
canv_feet_y = canv_y_bottom
canv_x_center = (x + offset_x) + w // 2

# 마스크 캔버스 배치
canv_body_mask = np.zeros((canvas_h, canvas_w), dtype=np.uint8)
shifted_contour = main_contour + [offset_x, offset_y]
cv2.drawContours(canv_body_mask, [shifted_contour], -1, 255, -1)
canv_line_mask = np.zeros((canvas_h, canvas_w), dtype=np.uint8)
canv_line_mask[offset_y:offset_y+img_h, offset_x:offset_x+img_w] = thinner_line_mask

# 4. 비디오 설정
fps = 48
frames_per_month = 3 
out = cv2.VideoWriter('cancer_final.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (canvas_w, canvas_h))

# 폰트 설정
try:
    if platform.system() == 'Darwin': font_path = "/System/Library/Fonts/Supplemental/AppleSDGothicNeo.ttc"
    elif platform.system() == 'Windows': font_path = "C:/Windows/Fonts/malgun.ttf"
    else: font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
    
    font_large = ImageFont.truetype(font_path, 60) 
    # [수정] 폰트 두 개로 분리 (숫자용: 큼, 일반용: 작음)
    font_num = ImageFont.truetype(font_path, 65) # 숫자 (큼)
    font_text = ImageFont.truetype(font_path, 50) # 나머지 글자 (보통)
except:
    font_large = font_num = font_text = ImageFont.load_default()

print("영상 생성 시작...")

for i in range(len(years_data)):
    target_year = years_data[i]
    start_rate_segment = rates_data[i]
    end_rate_segment = rates_data[i+1] if i < len(years_data) - 1 else rates_data[i]
    
    for month in range(1, 13):
        for f in range(frames_per_month):
            total_steps = 12 * frames_per_month
            current_step = (month - 1) * frames_per_month + f
            alpha = current_step / total_steps
            
            interpolated_rate = start_rate_segment + (end_rate_segment - start_rate_segment) * alpha
            percentage_text = interpolated_rate / 1000 
            
            height_calc_rate = max(min_rate_baseline, interpolated_rate)
            fill_ratio = (height_calc_rate - min_rate_baseline) / (max_rate_peak - min_rate_baseline)
            
            frame = np.ones((canvas_h, canvas_w, 3), dtype=np.uint8) * 255
            
            # 액체 채우기
            fill_height_px = int(fill_ratio * h_sil)
            curr_y_fill = canv_y_bottom - fill_height_px
            liquid_mask = np.zeros((canvas_h, canvas_w), dtype=np.uint8)
            liquid_mask[max(canv_y_top, curr_y_fill):canv_y_bottom, :] = 255
            final_fill_area = cv2.bitwise_and(canv_body_mask, liquid_mask)
            frame[final_fill_area > 0] = [70, 70, 230] 
            frame[canv_line_mask > 0] = [0, 0, 0]
            
            frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(frame_pil)
            
            # [수정 1] 머리 위 텍스트: "YYYY년 M 월에는"
            head_text = f"{target_year}년 {month}월에는"
            bbox_head = draw.textbbox((0, 0), head_text, font=font_large)
            head_w = bbox_head[2] - bbox_head[0]
            head_h = bbox_head[3] - bbox_head[1]
            head_text_x = canv_x_center - head_w // 2
            head_text_y = canv_y_top - head_h - 30
            draw.text((head_text_x, head_text_y), head_text, fill=(0, 0, 0), font=font_large)
            
            # [수정 2] 발 아래 문구: 숫자만 강조
            num_str = f"{percentage_text:.2f}%"
            rest_str = " 의 사람이 암에 걸렸습니다"
            
            # 각각 너비 계산
            bbox_num = draw.textbbox((0, 0), num_str, font=font_num)
            num_w = bbox_num[2] - bbox_num[0]
            
            bbox_rest = draw.textbbox((0, 0), rest_str, font=font_text)
            rest_w = bbox_rest[2] - bbox_rest[0]
            
            # 중앙 정렬 시작점 계산
            total_w = num_w + rest_w
            start_x = (canvas_w - total_w) // 2
            base_y = canv_feet_y + 40
            
            # 1. 숫자 그리기 (크고 굵게, 빨간색)
            # stroke_width=2를 줘서 볼드체 효과
            draw.text((start_x, base_y), num_str, fill=(200, 0, 0), font=font_num, stroke_width=2, stroke_fill=(200, 0, 0))
            
            # 2. 나머지 글자 그리기 (보통 크기, 검은색/회색)
            # 폰트 크기 차이만큼 Y 위치 살짝 조정
            text_y_adjust = base_y + (65 - 50) // 2 + 5 
            draw.text((start_x + num_w, text_y_adjust), rest_str, fill=(50, 50, 50), font=font_text)
            
            out.write(cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR))

out.release()
print("완성! 파일명: cancer_final.mp4")

영상 생성 시작...
완성! 파일명: cancer_final.mp4


In [21]:
import os

# 현재 폴더에 있는 파일 목록 출력
files = os.listdir()
print(f"현재 폴더: {os.getcwd()}")
print(f"찾은 파일들: {files}")

# 필수 파일 존재 여부 확인
required_files = ['cancer.csv', 'images.png']
for f in required_files:
    if f in files:
        print(f"✅ {f} 파일을 찾았습니다.")
    else:
        print(f"❌ {f} 파일이 없습니다! 주피터 노트북(.ipynb)과 같은 폴더로 옮겨주세요.")

현재 폴더: /Users/ihyun/Desktop/study
찾은 파일들: ['cancer_monthly_final.mp4', 'cancer_visual_final.mp4', '.DS_Store', 'cancer_slow_motion.mp4', 'cancer.csv', 'cancer_rising_final.mp4', 'image_2a4ba1.png', 'cancer_final_fix.mp4', 'cancer_rate_animation.mp4', 'cancer_relative_fill.mp4', '.ipynb_checkpoints', 'cancer_animaion.ipynb', 'cancer_final_animation.mp4', 'cancer_presentation_v2.mp4', 'images.png']
✅ cancer.csv 파일을 찾았습니다.
✅ images.png 파일을 찾았습니다.
